In [1]:
import numpy as np
import pandas as pd 
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import os

In [2]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/dataset-of-malicious-and-benign-webpages/Webpages_Classification_test_data.csv/Webpages_Classification_test_data.csv
/kaggle/input/dataset-of-malicious-and-benign-webpages/Webpages_Classification_train_data.csv/Webpages_Classification_train_data.csv


## Load dataset

In [13]:

def loadDataset(file_name):
    df = pd.read_csv(file_name)
    return df

df_train = loadDataset("/kaggle/input/dataset-of-malicious-and-benign-webpages/Webpages_Classification_train_data.csv/Webpages_Classification_train_data.csv")
df_test = loadDataset("/kaggle/input/dataset-of-malicious-and-benign-webpages/Webpages_Classification_test_data.csv/Webpages_Classification_test_data.csv")

## Data exploration

In [14]:
df_train.head()

,Unnamed: 0,url,url_len,ip_add,geo_loc,tld,who_is,https,js_len,js_obf_len,content,label
0,0,http://members.tripod.com/russiastation/,40,42.77.221.155,Taiwan,com,complete,yes,58.0,0.0,Named themselves charged particles in a manly ...,good
1,1,http://www.ddj.com/cpp/184403822,32,3.211.202.180,United States,com,complete,yes,52.5,0.0,And filipino field \n \n \n \n \n \n \n \n the...,good
2,2,http://www.naef-usa.com/,24,24.232.54.41,Argentina,com,complete,yes,103.5,0.0,"Took in cognitivism, whose adherents argue for...",good
3,3,http://www.ff-b2b.de/,21,147.22.38.45,United States,de,incomplete,no,720.0,532.8,fire cumshot sodomize footaction tortur failed...,bad
4,4,http://us.imdb.com/title/tt0176269/,35,205.30.239.85,United States,com,complete,yes,46.5,0.0,"Levant, also monsignor georges. In 1800, lists...",good


In [15]:
df_train.describe()

,Unnamed: 0,url_len,js_len,js_obf_len
count,1.200000e+06,1.200000e+06,1.200000e+06,1.200000e+06
mean,5.999995e+05,3.585337e+01,1.191463e+02,8.153424e+00
std,3.464103e+05,1.441089e+01,9.046649e+01,6.001398e+01
min,0.000000e+00,1.200000e+01,0.000000e+00,0.000000e+00
25%,2.999998e+05,2.600000e+01,6.650000e+01,0.000000e+00
50%,5.999995e+05,3.200000e+01,1.120000e+02,0.000000e+00
75%,8.999992e+05,4.200000e+01,1.580000e+02,0.000000e+00
max,1.199999e+06,7.210000e+02,8.541000e+02,8.028540e+02


In [16]:
df_train.iloc[3].content

'fire cumshot sodomize footaction tortur failed cra5h ribbed snigger assjockey jizz juggalo goddammit hitler bitchslap bisexual died cocknob skank drunk bong cigarette foursome ejaculating jiga blow angry fat felch lapdance sooty cocksuck fok sexpot pornking porno motherfuckin buttplug livesex titjob rimming butt sodomite mothafucka gook balllicker boonga williewanker pimp clamdiver hoser whitetrash manpaste cuntlicker mams mockey penetration mggor goddamit mocky vulva chinky pussylips turd wtf asspirate bullshit ethiopian beat-off scrotum peepshpw catholic ejaculated panti butt-fucker sluts spunky squaw niggardliness piss upskirt horseshit booty gipp nigerians boobs puss smut lucifer filipino cunn breastman titlicker swallow backdoor lesbin slut violence uk ass spic jijjiboo retarded blackout rentafuck sucker stripclub fistfuck easyslut knockers die shitoutofluck nigger robber fuckknob timbernigger argie pornography tonguetramp stupid russki asshole dyefly titbitnipply fuckmonkey moth

## Filter dataset

In [17]:
df_train_filtered = df_train[df_train.js_len > 100]

In [18]:
df_train_filtered = df_train_filtered[df_train.content.str.len() < 5000]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [19]:
df_train_filtered

,Unnamed: 0,url,url_len,ip_add,geo_loc,tld,who_is,https,js_len,js_obf_len,content,label
2,2,http://www.naef-usa.com/,24,24.232.54.41,Argentina,com,complete,yes,103.5,0.0,"Took in cognitivism, whose adherents argue for...",good
6,6,http://christian.net/,21,125.223.123.231,China,net,complete,yes,136.0,0.0,"Temperature variations, quantities is often ov...",good
8,8,http://www.greatestescapes.com,30,32.130.119.43,United States,com,complete,yes,183.0,0.0,"Working with run a. U.s., in their interpretat...",good
11,11,http://www.elsevier.com/locate/issn/0969-9961,45,30.109.197.177,United States,com,incomplete,yes,125.0,0.0,"Stadium. the the waters around iceland, the ir...",good
12,12,http://michel.swing.free.fr/,28,60.19.39.107,China,fr,complete,yes,174.0,0.0,Art may become aggressive. Exposition by terre...,good
...,...,...,...,...,...,...,...,...,...,...,...,...
1199992,1199992,http://www.rubiapigmentanaturalia.nl/,37,149.155.30.37,United Kingdom,nl,complete,yes,130.0,0.0,Dice to shops that catered to cigar workers. y...,good
1199993,1199993,http://www.kyrene.k12.az.us/schools/brisas/sun...,64,11.12.50.129,United States,k12.az.us,incomplete,no,153.5,0.0,Limited service william henry harrison for exp...,good
1199995,1199995,http://csrc.nist.gov/rbac/,26,62.120.245.128,Saudi Arabia,gov,complete,yes,106.0,0.0,There. this high gdp per capita of any other c...,good
1199997,1199997,http://www.syfyportal.com/news423380.html,41,181.240.45.113,Colombia,com,incomplete,yes,178.5,0.0,Both increase was deemed too imprecise to be b...,good


In [20]:
df_test_filtered = df_test[df_test.js_len > 100]

In [21]:
df_test_filtered = df_test_filtered[df_test.content.str.len() < 5000]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


## Vectorize with tf-idf

In [22]:
vectorizer = TfidfVectorizer(max_features=100, stop_words='english').fit(df_train.iloc[:50000].content)

## Test vectorizer

In [23]:
df_test_filtered

,Unnamed: 0,url,url_len,ip_add,geo_loc,tld,who_is,https,js_len,js_obf_len,content,label
1,1,http://www.collectiblejewels.com,32,188.120.171.121,Sweden,com,incomplete,yes,187.0,0.0,breast addict nudger whash ky darkie catholics...,good
3,3,http://www.mil.fi/maavoimat/kalustoesittely/00...,56,13.237.35.44,Australia,fi,complete,yes,152.0,0.0,Night being newton. according to the formation...,good
4,4,http://www.avclub.com/content/node/24539,40,220.193.62.89,China,com,complete,yes,150.0,0.0,34 per two children. if we exercise simple pra...,good
5,5,http://www.yellowstone.net/baptist,34,13.122.80.132,United States,net,complete,yes,140.5,0.0,"Own desert include interviews, first-hand obse...",good
8,8,http://www.newedgemanagement.com/,33,90.136.31.203,Sweden,com,complete,yes,108.0,0.0,"In 1953, the higher emotional intelligence. Or...",good
...,...,...,...,...,...,...,...,...,...,...,...,...
361923,361923,http://www.pbs.org/hopes/,25,65.66.200.110,United States,org,complete,yes,100.5,0.0,"Are existing has 3,300 members and independent...",good
361924,361924,http://www.esec.edu/,20,99.60.14.140,United States,edu,complete,yes,139.5,0.0,Press in assault from man and nature due to. G...,good
361928,361928,http://www.pulm.vcu.edu/,24,2.186.28.18,Iran,edu,incomplete,no,161.5,0.0,"All microscopic spaniards interbred, forming a...",good
361930,361930,http://members.shaw.ca/brianholden,34,217.114.245.202,Russia,ca,complete,yes,163.5,0.0,"Be mixed southeast alaska.. Corals, and 1936 s...",good


In [24]:
df_test_filtered.iloc[0].content

'breast addict nudger whash ky darkie catholics butthead bitcher loadedgun suckme pixy slave kock dickbrain bazongas niggled floo deepthroat assman fuckinright jesus kill willie cocksuck jesuschrist harder lovebone pornking flydye shitted buttman skankfuck knockers mggor dammit motherfucked slideitin fingerfuck fellatio honkey pimper mockey lezz jigga slav uterus pissoff paki whit slopy datnigga bollock republican hindoo hook fuckit motherfucking chickslick hitlerist felching period mams fuckedup whitetrash buttface bomd assassinate hork footfucker lesbayn assassin baptist sheeney barface thirdleg manpaste molestor kafir pooping fubar limey masturbating molest pubic bong roach snownigger fuckinnuts peck bootycall fetish sandnigger dumbfuck twobitwhore cum homicide butt dicklick triplex kike macaca incest niggaz kid fucked testicle buttfuck whiskydick whorefucker faeces cocklicker pooper fuckina nignog escort ballsack groe terrorist poverty skank aroused hymen crime syphilis brothel foo

In [33]:
vec_output = vectorizer.transform([df_test_filtered.iloc[0].content])

In [34]:
vec_output.toarray().shape

(1, 100)

In [35]:
vec_output.toarray()

array([[0.10941907, 0.        , 0.        , 0.48795847, 0.08845221,
        0.        , 0.08580228, 0.08884919, 0.10926052, 0.        ,
        0.        , 0.        , 0.        , 0.23991103, 0.15330569,
        0.        , 0.08781653, 0.0883208 , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.08868184,
        0.10875462, 0.1636833 , 0.0603014 , 0.        , 0.        ,
        0.08911616, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.08722453, 0.        , 0.        , 0.        ,
        0.10505878, 0.22938074, 0.12015347, 0.08846938, 0.17734064,
        0.10609776, 0.        , 0.06845462, 0.        , 0.08893024,
        0.        , 0.06909789, 0.        , 0.        , 0.        ,
        0.        , 0.12015347, 0.        , 0.13709897, 0.        ,
        0.        , 0.08858982, 0.05082465, 0.08732441, 0.08823534,
        0.27805952, 0.10936945, 0.08856398, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [36]:
vectorizer.get_feature_names()

['14',
 '20',
 '23',
 '97',
 'abc',
 'addition',
 'age',
 'alert',
 'basic',
 'break',
 'case',
 'close',
 'closed',
 'code',
 'comments',
 'concat',
 'condition',
 'confirm',
 'const',
 'constant',
 'continue',
 'date',
 'division',
 'document',
 'doe',
 'does',
 'equal',
 'eval',
 'execute',
 'executed',
 'firstname',
 'floor',
 'focus',
 'function',
 'german',
 'global',
 'greater',
 'history',
 'indexof',
 'input',
 'javascript',
 'john',
 'js',
 'lastindexof',
 'lastname',
 'left',
 'let',
 'line',
 'location',
 'log',
 'logical',
 'loop',
 'matching',
 'media',
 'methods',
 'multiple',
 'myscript',
 'number',
 'numbers',
 'objects',
 'onabort',
 'onerror',
 'open',
 'operations',
 'operator',
 'operators',
 'outside',
 'pattern',
 'perform',
 'person',
 'pi',
 'position',
 'prompt',
 'random',
 'right',
 'script',
 'scrollbars',
 'search',
 'self',
 'setinterval',
 'settimeout',
 'shift',
 'single',
 'slice',
 'src',
 'statement',
 'stop',
 'string',
 'strings',
 'textarea',
 'to

In [37]:
vec_train = vectorizer.transform(df_train.iloc[0:100000].content)

In [38]:
vec_train.toarray().shape

(100000, 100)

In [39]:
sorted(vec_output.toarray()[0])[-20:]

[0.10769133605583013,
 0.10875462128449373,
 0.10926051620273722,
 0.10936944868144002,
 0.10941906672134888,
 0.12015346680146989,
 0.12015346680146989,
 0.12018062357410639,
 0.1370989683312688,
 0.14835886067426043,
 0.1533056860051621,
 0.1636833037034844,
 0.17665870828090025,
 0.17734064443666922,
 0.21508803169101334,
 0.21721705841627434,
 0.22938074424796048,
 0.23991103078512294,
 0.27805951930365364,
 0.48795847069858894]

## Train logistic regression model

In [40]:
from sklearn.linear_model import LogisticRegression

# Regenerate vec_train if it's not defined
if 'vec_train' not in locals():
    vec_train = vectorizer.transform(df_train.iloc[0:100000].content)

# Initialize and train the Logistic Regression model
model = LogisticRegression()
model.fit(vec_train, df_train.iloc[0:100000].label)

LogisticRegression()

In [41]:
from sklearn.metrics import classification_report

# Transform the test data using the fitted vectorizer
vec_test = vectorizer.transform(df_test.content)

# Predict the labels for the test data
predictions = model.predict(vec_test)

# Generate and print the classification report
print(classification_report(df_test.label, predictions))

              precision    recall  f1-score   support

         bad       1.00      0.61      0.75      8062
        good       0.99      1.00      1.00    353872

    accuracy                           0.99    361934
   macro avg       0.99      0.80      0.87    361934
weighted avg       0.99      0.99      0.99    361934



In [45]:
def predict_unknown_data(url, content):
    vec_unknown = vectorizer.transform([content])
    prediction = model.predict(vec_unknown)
    return prediction[0]

unknown_url = "http://example.com/somepage"
unknown_content = "This is some example content to test the model."

predicted_label = predict_unknown_data(unknown_url, unknown_content)
print(f"The predicted label for the content from {unknown_url} is: {predicted_label}")

The predicted label for the content from http://example.com/somepage is: good
